In [1]:
from db import *
from sqlalchemy import insert, select
import json
DATA_PATH = './data'

# DB

In [2]:
with Session(engine) as session:
    session.add(Driver(name="Alani"))
    session.add(Driver(name="Audrey"))
    session.add(Driver(name="Cadence"))
    session.add(Driver(name="Daisy"))
    session.add(Driver(name="Mei Xi"))
    session.commit()

In [3]:
with Session(engine) as session:
    session.add(Buggy(name="Inviscid", abbreviation="inviscid"))
    session.add(Buggy(name="Kingping II", abbreviation="kp"))
    session.add(Buggy(name="Seraph", abbreviation="seraph"))
    session.add(Buggy(name="Mapambazuko", abbreviation="zuke"))
    session.commit()

In [4]:
file_list = ["zr/medialist.json", "ir/medialist.json"]
media_lists = {}
for file in file_list:
    with open(f'{DATA_PATH}/{file}') as f:
        media_list = json.load(f)['media']
        match file:
            case "zr/medialist.json":
                media_list = media_list[1:]
            case "ir/medialist.json":
                # Remove first 3 media for ir
                media_list = media_list[1:]
        media_list = sorted(media_list, key=lambda x: x['date'])
        media_lists[file] = media_list
        media_list = None
media_lists

{'zr/medialist.json': [{'type': 'video',
   'subtype': 'video',
   'url': 'http://172.26.203.135/DCIM/100_VIRB/VIRB0121.MP4',
   'thumbUrl': 'http://172.26.203.135/DCIM/100_VIRB/VIRB0121.THM',
   'fav': 'false',
   'lowResVideoPath': 'http://172.26.203.135/DCIM/100_VIRB/VIRB0121.GLV',
   'fitURL': 'http://172.26.203.135/GMetrix/2025-09-21-07-10-41.fit',
   'name': 'VIRB0121.MP4',
   'fileSize': 784192855,
   'date': 1758453042,
   'duration': 208.0},
  {'type': 'video',
   'subtype': 'video',
   'url': 'http://172.26.203.135/DCIM/100_VIRB/VIRB0122.MP4',
   'thumbUrl': 'http://172.26.203.135/DCIM/100_VIRB/VIRB0122.THM',
   'fav': 'false',
   'lowResVideoPath': 'http://172.26.203.135/DCIM/100_VIRB/VIRB0122.GLV',
   'fitURL': 'http://172.26.203.135/GMetrix/2025-09-21-07-27-14.fit',
   'name': 'VIRB0122.MP4',
   'fileSize': 798959808,
   'date': 1758454036,
   'duration': 212.0},
  {'type': 'video',
   'subtype': 'video',
   'url': 'http://172.26.203.135/DCIM/100_VIRB/VIRB0123.MP4',
   'th

In [5]:
with Session(engine) as session:
    session.add(RollDate(year=2025, month=9, day=21, type=RollType.WEEKEND))
    session.add(RollDate(year=2025, month=11, day=2, type=RollType.WEEKEND))
    session.commit()

In [6]:
with Session(engine) as session:
    session.add(Sensor(type="virb", name="Zuke R", abbreviation="zr"))
    session.add(Sensor(type="virb", name="Inviscid R", abbreviation="ir"))
    session.commit()

In [7]:
def add_roll(session: Session, media: dict, 
             roll_date: RollDate, driver: Driver, buggy: Buggy, sensor: Sensor, 
             roll_num: int, use_thm: bool = False):
    dt = datetime.fromtimestamp(media['date'])
    roll = Roll(roll_date_id=roll_date.id, 
                driver_id=driver.id,
                buggy_id=buggy.id,
                roll_number=roll_num,
                start_time=dt)
    session.add(roll)
    session.flush() # populates roll.id
    vid = media['lowResVideoPath'].split('/')[-1].split('.')[0]
    vid = f"%videos%/{sensor.abbreviation}/{vid}.GLV"
    session.add(RollFile(roll_id=roll.id,
                         sensor_id=sensor.id,
                         type='video_preview',
                         uri=vid))
    
    fit = media['fitURL'].split('/')[-1].split('.')[0]
    fit = f"%fit%/{sensor.abbreviation}/{fit}.fit"
    session.add(RollFile(roll_id=roll.id,
                         sensor_id=sensor.id,
                         type='fit',
                         uri=fit))
    if use_thm:
        thm = media['thumbUrl'].split('/')[-1].split('.')[0]
        thm = f"%thumbnails%/{sensor.abbreviation}/{thm}.jpg"
        session.add(RollFile(roll_id=roll.id,
                            sensor_id=sensor.id,
                            type='thumbnail',
                            uri=thm))
        
    return roll

In [8]:
roll_nums = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
with Session(engine) as session:
    for file in file_list:
        i = 0
        media_list = media_lists[file]
        match file:
            case "zr/medialist.json":
                roll_date = session.scalars(
                    select(RollDate).where(
                        RollDate.year == 2025, #type: ignore
                        RollDate.month == 9, #type: ignore
                        RollDate.day == 21 #type: ignore
                    ) 
                ).one() 
                
                sensor = session.scalars(
                    select(Sensor)
                        .where(Sensor.abbreviation == "zr") #type: ignore
                ).one()
                thm = False
            case "ir/medialist.json":
                roll_date = session.scalars(
                    select(RollDate).where(
                        RollDate.year == 2025, #type: ignore
                        RollDate.month == 11, #type: ignore
                        RollDate.day == 2 #type: ignore
                    ) 
                ).one() 
                
                sensor = session.scalars(
                    select(Sensor)
                        .where(Sensor.abbreviation == "ir") #type: ignore
                ).one()
                thm = True
        driver = session.scalars(select(Driver)
                                .where(Driver.name == "Mei Xi")).one() #type: ignore
        buggy = session.scalars(select(Buggy)
                                .where(Buggy.abbreviation == "inviscid")).one() #type: ignore
        for media in media_list:
            r = add_roll(session, media, roll_date, driver, buggy, sensor, roll_nums[i % len(roll_nums)], thm)
            print(r)
            i += 1
    # session.rollback()
    session.commit()

driver_id=5 buggy_id=1 roll_date_id=1 roll_number=1 start_time=datetime.datetime(2025, 9, 21, 7, 10, 42) created_at=datetime.datetime(2025, 11, 10, 0, 24, 34, 236649, tzinfo=datetime.timezone.utc) updated_at=datetime.datetime(2025, 11, 10, 0, 24, 34, 236659, tzinfo=datetime.timezone.utc) id=1 driver_notes='' mech_notes='' pusher_notes=''
driver_id=5 buggy_id=1 roll_date_id=1 roll_number=2 start_time=datetime.datetime(2025, 9, 21, 7, 27, 16) created_at=datetime.datetime(2025, 11, 10, 0, 24, 34, 238404, tzinfo=datetime.timezone.utc) updated_at=datetime.datetime(2025, 11, 10, 0, 24, 34, 238410, tzinfo=datetime.timezone.utc) id=2 driver_notes='' mech_notes='' pusher_notes=''
driver_id=5 buggy_id=1 roll_date_id=1 roll_number=3 start_time=datetime.datetime(2025, 9, 21, 7, 43, 36) created_at=datetime.datetime(2025, 11, 10, 0, 24, 34, 240929, tzinfo=datetime.timezone.utc) updated_at=datetime.datetime(2025, 11, 10, 0, 24, 34, 240935, tzinfo=datetime.timezone.utc) id=3 driver_notes='' mech_notes

In [ ]:
with Session(engine) as session:
    roll_date = session.scalars(
        select(RollDate).where(
            RollDate.year == 2025,
            RollDate.month == 9,
            RollDate.day == 21
        )
    )
    print(roll_date.one())

year=2025 created_at=datetime.datetime(2025, 11, 10, 0, 5, 12, 847663) day=21 temperature=None type=<RollType.WEEKEND: 'weekend'> updated_at=datetime.datetime(2025, 11, 10, 0, 5, 12, 847671) id=1 month=9 notes='' humidity=None


In [ ]:
for roll in media_list:
    vid = roll['lowResVideoPath'].split('/')[-1].split('.')[0]
    vid = f"videos/zuke_r/{vid}.GLV"
    # thum = roll['thumbUrl'].split('/')[-1].split('.')[0]
    fit = roll['fitURL'].split('/')[-1].split('.')[0]
    fit = f"backend/notebooks/data/zr/{fit}.fit"
    print(vid, fit)

videos/zuke_r/VIRB0120.GLV backend/notebooks/data/zr/2025-09-20-08-19-55.fit
videos/zuke_r/VIRB0121.GLV backend/notebooks/data/zr/2025-09-21-07-10-41.fit
videos/zuke_r/VIRB0122.GLV backend/notebooks/data/zr/2025-09-21-07-27-14.fit
videos/zuke_r/VIRB0123.GLV backend/notebooks/data/zr/2025-09-21-07-43-36.fit
videos/zuke_r/VIRB0124.GLV backend/notebooks/data/zr/2025-09-21-07-56-07.fit
videos/zuke_r/VIRB0125.GLV backend/notebooks/data/zr/2025-09-21-08-12-58.fit
videos/zuke_r/VIRB0126.GLV backend/notebooks/data/zr/2025-09-21-08-26-18.fit
videos/zuke_r/VIRB0127.GLV backend/notebooks/data/zr/2025-09-21-08-51-38.fit


In [ ]:
with Session(engine) as session:
    d = session.scalars(select(Driver)
                        .where(Driver.name == "Alani")).one() #type: ignore
d

Driver(created_at=datetime.datetime(2025, 11, 7, 15, 31, 43, 289931), id=1, name='Alani', updated_at=datetime.datetime(2025, 11, 7, 15, 31, 43, 290041))

In [ ]:
# with get_connection() as conn:
#     conn.execute(insert(driver_table).values(name="Alani"))
#     conn.execute(insert(driver_table).values(name="Audrey"))
#     conn.execute(insert(driver_table).values(name="Cadence"))
#     conn.execute(insert(driver_table).values(name="Daisy"))
#     conn.execute(insert(driver_table).values(name="Mei Xi"))
#     conn.commit()

In [ ]:
# with get_connection() as conn:
#     result = conn.execute(driver_table.select())
#     for row in result:
#         print(row)

# Download

In [ ]:
import json
import requests
from tqdm.notebook import tqdm

with open('./data/ir_11_2/medialist.json') as f:
    media_list = json.load(f)['media'][4:]
media_list

[{'type': 'video',
  'subtype': 'video',
  'url': 'http://172.26.203.128/DCIM/101_VIRB/VIRB0123.MP4',
  'thumbUrl': 'http://172.26.203.128/DCIM/101_VIRB/VIRB0123.THM',
  'fav': 'false',
  'lowResVideoPath': 'http://172.26.203.128/DCIM/101_VIRB/VIRB0123.GLV',
  'fitURL': 'http://172.26.203.128/GMetrix/2025-11-02-07-28-52.fit',
  'name': 'VIRB0123.MP4',
  'fileSize': 820862362,
  'date': 1762086532,
  'duration': 218.0},
 {'type': 'video',
  'subtype': 'video',
  'url': 'http://172.26.203.128/DCIM/101_VIRB/VIRB0124.MP4',
  'thumbUrl': 'http://172.26.203.128/DCIM/101_VIRB/VIRB0124.THM',
  'fav': 'false',
  'lowResVideoPath': 'http://172.26.203.128/DCIM/101_VIRB/VIRB0124.GLV',
  'fitURL': 'http://172.26.203.128/GMetrix/2025-11-02-07-44-48.fit',
  'name': 'VIRB0124.MP4',
  'fileSize': 1052645829,
  'date': 1762087490,
  'duration': 279.0},
 {'type': 'video',
  'subtype': 'video',
  'url': 'http://172.26.203.128/DCIM/101_VIRB/VIRB0125.MP4',
  'thumbUrl': 'http://172.26.203.128/DCIM/101_VIRB/

In [ ]:
for media in tqdm(media_list):
    url = media['lowResVideoPath']
    # Stream video to file
    response = requests.get(url)
    filename = url.split('/')[-1].replace('GLV', 'mp4')
    with open(f'../../../videos/ir_11_2/{filename}', 'wb') as f:
        f.write(response.content)

  0%|          | 0/6 [00:00<?, ?it/s]